In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

for path in sys.path:
    print(path)

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC




from src.setup_problem import Setup
# from src.setup_problem_loss import Setup
from src.misc import visualize_result

import cvxpy as cp

from src.misc import *

/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages
/home/onoue/ws/lukasiewicz_1
/home/onoue/ws/lukasiewicz_1


# データの準備

In [66]:
# train_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/train_index.csv", index_col=0)["0"].to_list()
# # display(train_index)

# test_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/test_index.csv", index_col=0)["0"].to_list()
# # display(test_index)

In [40]:
data = pd.read_csv("./data/diabetes_discretized.csv", index_col=0)

feature_names = data.columns.to_list()[1:]

X = data.drop(["Outcome"], axis=1)
y = data["Outcome"]

random_state = 45
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=test_size,
                                                    random_state=random_state)

train_index = X_train.index
test_index = X_test.index

import os
project_dir_path = "/home/onoue/ws/lukasiewicz_1/"
data_dir_path = "inputs/pima_indian_diabetes"
save_dir_path = os.path.join(project_dir_path, data_dir_path)

file_path_1 = os.path.join(save_dir_path, 'train_index_3.csv')
pd.DataFrame(train_index).to_csv(file_path_1)

file_path_2 = os.path.join(save_dir_path, 'test_index_3.csv')
pd.DataFrame(test_index).to_csv(file_path_2)


X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# SVM（線形カーネル）のモデルを作成
svm_linear = SVC(kernel='linear')

# モデルを訓練
svm_linear.fit(X_train, y_train)

# テストデータで予測
y_pred_linear = svm_linear.predict(X_test)

# Accuracyを計算
accuracy_linear = accuracy_score(y_test, y_pred_linear)

# Confusion Matrixを計算
cm_linear = confusion_matrix(y_test, y_pred_linear)

# Classification Reportを出力
report_linear = classification_report(y_test, y_pred_linear)

# 結果を出力
print("SVM (Linear Kernel) Accuracy:", accuracy_linear)
print()
print()
print("SVM (Linear Kernel) Confusion Matrix:\n", cm_linear)
print()
print()
print("SVM (Linear Kernel) Classification Report:\n", report_linear)

SVM (Linear Kernel) Accuracy: 0.7835820895522388


SVM (Linear Kernel) Confusion Matrix:
 [[85  9]
 [20 20]]


SVM (Linear Kernel) Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.90      0.85        94
           1       0.69      0.50      0.58        40

    accuracy                           0.78       134
   macro avg       0.75      0.70      0.72       134
weighted avg       0.77      0.78      0.77       134



In [42]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[train_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/train_3')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


for feature_name in feature_names:
    df = features.copy()
    df['target'] = df[feature_name].replace(0, -1)
    # display(df)

    file_name = "L_" + feature_name + '.csv'
    file_path = os.path.join(data_dir_path, file_name)
    df.to_csv(file_path)


data_num = 15
data_dim = 21

arr_U = np.random.randint(2, size=(data_num, data_dim))
df_U = pd.DataFrame(arr_U)
df_U.to_csv(os.path.join(data_dir_path, 'U.csv'))

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
47,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,0,-1
423,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
24,0,0,1,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
739,1,0,0,0,1,0,0,1,0,1,...,0,0,1,1,0,0,0,1,0,1
85,1,0,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0,1,0,0,0,1,0,1,0,1,...,0,1,0,1,0,0,1,0,0,1
704,1,0,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
631,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
741,1,0,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,1,0,0,-1


In [43]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[test_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/test_3')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)

 
df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


# for feature_name in feature_names:
#     df = features.copy()
#     df['target'] = df[feature_name].replace(0, -1)
#     # display(df)

#     file_name = "L_" + feature_name + '.csv'
#     file_path = os.path.join(data_dir_path, file_name)
#     df.to_csv(file_path)

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
227,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
5,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
646,1,0,0,0,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,1
381,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,-1
268,1,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,0,1,0,0,0,1,0,1,0,1,...,1,0,0,0,0,1,0,1,0,1
595,1,0,0,0,0,1,0,1,0,1,...,0,1,0,0,1,0,1,0,0,1
190,1,0,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
421,1,0,0,1,0,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,-1


In [5]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')

train_data_dir_path = os.path.join(data_dir_path, "train_5")

file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules']
}

problem_instance = Setup(train_data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.06902694702148438 seconds! 

Loading rules ...
Done in 0.0002162456512451172 seconds! 

Identifying predicates ...
Done in 0.0008831024169921875 seconds! 

Constructing objective function ...
Done in 2.8779046535491943 seconds! 

Constructing constraints ...
Done in 13.160948514938354 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 26 02:41:48 PM: Your problem has 12322 variables, 37416 constraints, and 0 parameters.
(CVXPY) Nov 26 02:41:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 26 02:41:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 26 02:41:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 26 02:41:55 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 26 02:41:55 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

In [8]:
test_data_dir_path = os.path.join(data_dir_path, "test_5")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value

y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)

accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)


,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
227,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
5,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
646,1,0,0,0,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,1
381,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,-1
268,1,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,-1


Accuracy: 
0.7835820895522388

Confusion matrix: 
[[85  9]
 [20 20]]

classification report: 
              precision    recall  f1-score   support

          -1       0.81      0.90      0.85        94
           1       0.69      0.50      0.58        40

    accuracy                           0.78       134
   macro avg       0.75      0.70      0.72       134
weighted avg       0.77      0.78      0.77       134



In [9]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train_4")
file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules']
}



problem_instance = Setup(train_data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.059507131576538086 seconds! 

Loading rules ...
Done in 0.0002048015594482422 seconds! 

Identifying predicates ...
Done in 0.0008592605590820312 seconds! 

Constructing objective function ...
Done in 2.768888473510742 seconds! 

Constructing constraints ...
Done in 18.200183391571045 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 26 02:49:20 PM: Your problem has 12014 variables, 38096 constraints, and 0 parameters.
(CVXPY) Nov 26 02:49:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 26 02:49:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 26 02:49:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 26 02:49:27 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 26 02:49:27 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

In [11]:
test_data_dir_path = os.path.join(data_dir_path, "test_4")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

y_pred = p_dict['Outcome'](X_test).value
y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)


accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,target
419,3,129,64,29,26.4,0.219,28,1
186,8,181,68,36,30.1,0.615,60,1
556,1,97,70,40,38.1,0.218,30,-1
738,2,99,60,17,36.6,0.453,21,-1
320,4,129,60,12,27.5,0.527,31,-1


Accuracy: 
0.6716417910447762

Confusion matrix: 
[[90  0]
 [44  0]]

classification report: 
              precision    recall  f1-score   support

        -1.0       0.67      1.00      0.80        90
         1.0       0.00      0.00      0.00        44

    accuracy                           0.67       134
   macro avg       0.34      0.50      0.40       134
weighted avg       0.45      0.67      0.54       134



/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

: 